In [1]:
import math
import sys
import numpy as np
from datetime import datetime
import os
import csv
from tqdm import tqdm
import re
from some_func import util
from math import atan, pi, log, tan, exp

In [2]:
# 网格类
class Grid:
    def __init__(self):
        self.pickup_points = []  # 位于当前网格的定位点
        self.locating_of_grid = [] # 网格的位置，里面所有点的经纬度平均

In [3]:
def Distance_of_two_points(pointa,pointb):
    pointa_mercator = util.wgs84_to_mercator(pointa[0],pointa[1])
    pointb_mercator = util.wgs84_to_mercator(pointb[0],pointb[1])
    distance = ((pointa_mercator[0] - pointb_mercator[0]) ** 2 + (pointa_mercator[1] - pointb_mercator[1]) ** 2) ** 0.5
    return distance

In [4]:
# def wgs84_to_mercator(lon, lat):
#     x = lon * 20037508.342789 / 180
#     y = log(tan((90 + lat) * pi / 360)) / (pi / 180)
#     y = y * 20037508.34789 / 180
#     return [x, y]

In [5]:
def get_pickup_point(traj_data: dict):
    """

    Args:
        traj_data: 轨迹数据


    Returns:上下车的点的定位点

    """
    pickup_point_list = []
    for key,value in tqdm(traj_data.items()):
        # pickup_point_index = []
        coordinates = value[1]
        status_list = value[2]
        for i in range(1,len(coordinates)):
            if status_list[i] != status_list[i-1]:
                pickup_point_list.append(coordinates[i])

    return pickup_point_list

In [6]:
def load_data():# data:key->taxi_id,value->[[time],[coordinates(not mercator)],[status]],
    data = {}
    lon_min = 180
    lon_max = 0
    lat_min = 90
    lat_max = 0

    file_need_remove = []
    file_need_stay = []
    traj_folder = os.path.join(os.getcwd(), "traj")
    for file_name in tqdm(os.listdir(traj_folder)) :
        if file_name.startswith("traj_") and file_name.endswith(".csv"):
            match = re.search(r"traj_(\d+)\.csv", file_name)
            taxi_id = match.group(1)
            file_path = os.path.join(traj_folder, file_name)
            # data_illegal_count = 0

            time_list = []
            coordinates = []
            status_list = []

            if_use_this_data  = True

            with open(file_path, mode='r', encoding='utf-8') as csv_file:
                csv_reader = csv.DictReader(csv_file)
                for row in csv_reader:
                    ID = int(row['ID'])
                    time_str = row['Time']
                    time = datetime.strptime(time_str, "%H:%M:%S").time()  # 转换为 time 对象
                    longitude = float(row['Longitude'])
                    latitude = float(row['Latitude'])
                    status = int(row['Status'])

                    # 检测到非法数据就弃用该出租车的数据
                    if not (113.75 < longitude < 114.63):
                        if_use_this_data  = False
                        break
                    if not (22.45 < latitude < 22.85):
                        if_use_this_data  = False
                        break
                    if status != 1 and status != 0 :
                        if_use_this_data  = False
                        break


                    time_list.append(time)
                    coordinates.append([longitude,latitude])
                    status_list.append(status)


                    # lon_min = min(lon_min,longitude)
                    # lon_max = max(lon_max,longitude)
                    # lat_min = min(lat_min,latitude)
                    # lat_max = max(lat_max,latitude)

                    # data.setdefault(ID, [[], [], []])
                    # data[ID][0].append(time)
                    # data[ID][1].append([longitude,latitude])
                    # data[ID][1].append(util.mercator_to_wgs84(longitude, latitude))
                    # data[ID][2].append(status)
                    # print(ID)
                    # print(time,type(time))
                    # print([longitude, latitude])
                    # print(status,type(status))
                    # print("----------------------")

            if if_use_this_data:
                data[taxi_id] = [time_list,coordinates,status_list]
                lon_min = min(lon_min,min(coord[0] for coord in coordinates))
                lon_max = max(lon_max,max(coord[0] for coord in coordinates))
                lat_min = min(lat_min,min(coord[1] for coord in coordinates))
                lat_max = max(lat_max,max(coord[1] for coord in coordinates))
                file_need_stay.append(file_name)
            else:
                file_need_remove.append(file_name)


                # if data_illegal_count > 2:
                #     file_need_remove.append(file_name)
    outputfile = "file_need_stay.txt"
    print(len(file_need_stay))
    with open(outputfile, mode='w', encoding='utf-8') as f:
        for filename in file_need_stay:
            f.write(filename + "\n")  # 每个文件名写入一行
    outputfile = "file_need_remove.txt"
    with open(outputfile, mode='w', encoding='utf-8') as f:
        for filename in file_need_remove:
            f.write(filename + "\n")  # 每个文件名写入一行
    return data,[lon_min,lon_max,lat_min,lat_max]

In [7]:
# 根据经纬度最大小值以及网格大小确定区域被分为rows*cols数量的网格
def get_grids_i_j(x_min_y_min_mercator,x_max_y_max_mercator,grid_size):
    x_length = x_max_y_max_mercator[0] - x_min_y_min_mercator[0]
    y_length = x_max_y_max_mercator[1] - x_min_y_min_mercator[1]
    cols = math.ceil(x_length / grid_size)
    rows = math.ceil(y_length / grid_size)
    return cols,rows

In [8]:
# 获得一个上下车点在区域中所在网格的索引
def get_i_j_of_pickup_point(x_min_y_min_mercator,x_max_y_max_mercator,grid_size,pickup_point):
    pickup_point_mercator = util.wgs84_to_mercator(pickup_point[0],pickup_point[1])
    x_dis = pickup_point_mercator[0] - x_min_y_min_mercator[0]
    y_dis = pickup_point_mercator[1] - x_min_y_min_mercator[1]
    i = math.floor(x_dis / grid_size)
    j = math.floor(y_dis / grid_size)
    # 检查边界是否超出区域
    if i < 0 or j < 0 or pickup_point_mercator[0] > x_max_y_max_mercator[0] or pickup_point_mercator[1] > x_max_y_max_mercator[1]:
        raise ValueError("Pickup point is outside the grid region.")
    return i,j

In [9]:
def get_Grid_locating_point(grid:Grid):
    lon_sum = 0
    lat_sum = 0
    points = grid.pickup_points
    for point in points:
        lon_sum += point[0]
        lat_sum += point[1]
    return [lon_sum/len(points),lat_sum/len(points)]

In [10]:
# 参数

# lon经度 lat纬度
# x,y mercator


k = 20# 网格的大小，单位为米
lameda = 20 # 网格密度阈值
fai = 1.3 # 定义9里面那个参数
limit = fai*k
print(limit)

26.0


In [11]:
traj_data,district = load_data()
print(district)

100%|██████████| 14728/14728 [07:55<00:00, 30.94it/s]

14093
[113.752022, 114.567619, 22.456734, 22.8493]


In [12]:
# # 这段是拿来把原本的traj文件夹里面的数据分割成两部分的，不用管
#
#
#
#
# import os
# import shutil
#
# # 定义路径
# current_dir = os.getcwd()  # 获取当前目录
# traj_folder = os.path.join(current_dir, "traj")
# traj_remove_folder = os.path.join(current_dir, "traj_remove")
# file_need_remove = os.path.join(current_dir, "file_need_remove.txt")
#
# # 创建 traj_remove 文件夹（如果不存在）
# if not os.path.exists(traj_remove_folder):
#     os.makedirs(traj_remove_folder)
#
# # 读取 file_need_remove.txt
# if os.path.exists(file_need_remove):
#     with open(file_need_remove, mode="r", encoding="utf-8") as f:
#         filenames = [line.strip() for line in f.readlines() if line.strip()]
# else:
#     print(f"文件 {file_need_remove} 不存在！")
#     exit(1)
#
# # 移动文件
# for filename in filenames:
#     source_path = os.path.join(traj_folder, filename)
#     target_path = os.path.join(traj_remove_folder, filename)
#
#     if os.path.exists(source_path):
#         shutil.move(source_path, target_path)  # 移动文件
#         print(f"已移动: {filename}")
#     else:
#         print(f"文件 {filename} 不存在于 {traj_folder} 中！")
#
# print("处理完成。")

记录，第一次
```
if not (113 < longitude < 114.6):
    if_use_this_data  = False
    break
if not (22.1 < latitude < 22.7):
    if_use_this_data  = False
    break
```
8975条轨迹被保留
5753条轨迹被移除
[113.310249, 114.567619, 22.157217, 22.699966]

记录，第二次
```
if not (113.4 < longitude < 114.4):
    if_use_this_data  = False
    break
if not (22.15 < latitude < 22.6):
    if_use_this_data  = False
    break
```
184条轨迹被保留
14544条轨迹被移除


if not (113.75 < longitude < 114.63):
    if_use_this_data  = False
    break
if not (22.45 < latitude < 22.85):
    if_use_this_data  = False
    break



In [13]:
count = 0
for key,value in traj_data.items():
    count += len(value[1])
print(count)

44943691


In [14]:
pickup_points = get_pickup_point(traj_data)



# traj_data = []



print(len(pickup_points))

100%|██████████| 14093/14093 [00:03<00:00, 4275.94it/s]

1040274


In [15]:
# 才发现网格的lon_max啥的是上下车点的lon_max,
lon_min = min([pickup_point[0] for pickup_point in pickup_points])
lon_max = max([pickup_point[0] for pickup_point in pickup_points])
lat_min = min([pickup_point[1] for pickup_point in pickup_points])
lat_max = max([pickup_point[1] for pickup_point in pickup_points])
print(lon_min,lon_max,lat_min,lat_max)
x_min_y_min = util.wgs84_to_mercator(lon_min,lat_min)
x_max_y_max = util.wgs84_to_mercator(lon_max,lat_max)

113.75502 114.567215 22.459433 22.843834


In [16]:
# 创建网格，将网格四个点映射为mercator坐标，获得四个点坐标，右经度-左经度获得长度，长度/k,向上取整，得到网格的横的数量
# 同理得到网格竖的数量，创建grids[]，将点映射到网格时，以左下角为准，比如说一个点属于网格[i][j],以左下角为准

rows,cols = get_grids_i_j(x_min_y_min,x_max_y_max,k)
print(rows,cols)

grids = [[Grid() for _ in range(cols)] for _ in range(rows)]

# 遍历上下车点，将其映射到grid，添加进grid
for pickup_point in tqdm(pickup_points) :
    i,j = get_i_j_of_pickup_point(x_min_y_min,x_max_y_max,k,pickup_point)
    grids[i][j].pickup_points.append(pickup_point)



# pickup_points = []



# 计算每个grid的点数量，超过lameda就计算平均位置，记录index
G = {}
count_hot_point = 0
for row in range(rows):
    for col in range(cols) :
        if len(grids[row][col].pickup_points) > lameda:
            # 计算平均位置
            grids[row][col].locating_of_grid = get_Grid_locating_point(grids[row][col])

            G[count_hot_point] = [row,col]
            count_hot_point += 1

            # grid_index.append([[row,col], len(grids[row][col].pickup_points)])


print(count_hot_point)
print(len(G))

4521 2319


100%|██████████| 1040274/1040274 [00:04<00:00, 235434.93it/s]


8837
8837


In [17]:
# for value in G.values():
#     print(value)
outputfile = "file_grid_locate_1.txt"
with open(outputfile, mode='w', encoding='utf-8') as f:
    for point in G.values():
        f.write(str(grids[point[0]][point[1]].locating_of_grid) + "\n")

In [18]:
# count = 0
# O= []
# while G:
#     hot_district = []
#     key,index = next(iter(G.items()))# 获得G中的一个元素
#     hot_district.append(index)
#     # hot_district.append(grids[index[0]][index[1]].locating_of_grid)
#     del G[key]
#     keys_to_remove = []
#     if G:
#         for key1,g in G.items():
#             for h in hot_district:
#                 pointa = grids[g[0]][g[1]].locating_of_grid
#                 pointb = grids[h[0]][h[1]].locating_of_grid
#                 if Distance(pointa,pointb) < fai*k :
#                     hot_district.append(pointa)
#                     keys_to_remove.append(key1)
#                     break
#         for key in keys_to_remove:
#             del G[key]
#     O.append(hot_district)
#     count += 1
#
# for hot_district in O:
#     print(hot_district)

In [19]:
# count = 0
# O = []
#
# while G:
#     hot_district = []
#
#     # 获取字典中的第一个元素
#     key, index_of_a_grid = next(iter(G.items()))
#     hot_district.append(index_of_a_grid)
#     del G[key]
#
#     # 创建待删除的键列表，避免并发修改字典
#     keys_to_remove = []
#
#     print(f"1.{len(G)}")
#
#     if G:
#         for key1, g in G.items():
#             for h in hot_district:
#                 pointa = grids[g[0]][g[1]].locating_of_grid
#                 pointb = grids[h[0]][h[1]].locating_of_grid
#                 if Distance_of_two_points(pointa, pointb) < fai*k:
#                     hot_district.append(g)
#                     keys_to_remove.append(key1)
#
#         # 删除符合条件的键
#         for key in keys_to_remove:
#             del G[key]
#
#     # 保存当前分组
#     O.append(hot_district)
#     count += 1
#
#     print(f"2.{len(G)}")
#     print("--------------------")
#
# # 输出分组结果
# print(count)
# for hot_district in O:
#     print(hot_district)


In [20]:
# for value in G.values():
#     print(value)

In [21]:
# O = []
#
# while G:
#     hot_district = []
#
#     # 获取字典中的第一个元素
#     key, index_of_a_grid = next(iter(G.items()))
#     hot_district.append(index_of_a_grid)
#     del G[key]
#
#     # 创建待删除的键列表，避免并发修改字典
#     keys_to_remove = []
#
#     # print(f"1.{len(G)}")
#
#     if G:
#         for key1, g in G.items():
#             for h in hot_district:
#                 pointa = grids[g[0]][g[1]].locating_of_grid
#                 pointb = grids[h[0]][h[1]].locating_of_grid
#                 if Distance_of_two_points(pointa, pointb) < fai*k:
#                     hot_district.append(g)
#                     keys_to_remove.append(key1)
#                     break
#
#         # 删除符合条件的键
#         for key in keys_to_remove:
#             del G[key]
#
#     # 保存当前分组
#     O.append(hot_district)
#     count += 1
#
#     # print(f"2.{len(G)}")
#     # print("--------------------")
#
# # 输出分组结果
# print(count)
# for hot_district in O:
#     print(hot_district)